# Test Training with ResNet50


In [2]:
import os
import requests
import tarfile
from tqdm import tqdm  # for progress bar
from IPython.display import display, Image



In [5]:
# Function to download a file from a URL with a custom progress bar
def download_file(url, save_path):
    with requests.get(url, stream=True) as response:
        response.raise_for_status()
        total_size = int(response.headers.get('content-length', 0))
        progress = tqdm(total=total_size, unit='B', unit_scale=True, desc=os.path.basename(save_path), ncols=80)
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    progress.update(len(chunk))
        progress.close()
        

# URLs for ImageNet dataset files (replace with actual download links)
train_url = 'http://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train.tar'
val_url = 'http://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar'

# Directory to save dataset files
dataset_dir = '/home/ec2-user/SageMaker/imagenet_dataset'

# Create directory if it doesn't exist
os.makedirs(dataset_dir, exist_ok=True)


# Download train dataset
train_tar_path = os.path.join(dataset_dir, 'ILSVRC2012_img_train.tar')
print(f"Downloading train dataset from {train_url}...")
download_file(train_url, train_tar_path)



# Download validation dataset
val_tar_path = os.path.join(dataset_dir, 'ILSVRC2012_img_val.tar')
print(f"Downloading validation dataset from {val_url}...")
download_file(val_url, val_tar_path)


# Extract train dataset
print(f"Extracting train dataset to {dataset_dir}...")
with tarfile.open(train_tar_path, 'r') as tar:
    tar.extractall(dataset_dir)

    
# Extract validation dataset
print(f"Extracting validation dataset to {dataset_dir}...")
with tarfile.open(val_tar_path, 'r') as tar:
    tar.extractall(dataset_dir)

print("Dataset extraction complete!")



In [ ]:
////////////////////////////////////////////////////////////////////////////////////////////////


In [3]:
import os
import tarfile
from tqdm import tqdm
import time
import shutil
import tensorflow as tf


# Define the target synset IDs and their corresponding animal names
target_species_synsets = {
    'n02129165': 'lion',
    'n02129604': 'tiger',
    'n02391049': 'zebra',
    'n02504013': 'elephant',
    'n02398521': 'hippo',
    'n02138441': 'meerkat',
    'n02480855': 'gorilla',
    'n02056570': 'king_penguin',
    'n02130308': 'cheetah',
    'n02509815': 'red_panda',
    'n02133161': 'black_bear',
    'n02497673': 'lemur',
    'n02007558': 'flamingo',
    'n02481823': 'chimpanzee',
    'n02480495': 'orangutan',
    'n02128925': 'jaguar',
    'n02510455': 'panda',
    'n02454379': 'armadillo',
    'n02117135': 'hyena',
    'n01644373': 'tree_frog'
}

def extract_images_to_folders(dataset_dir, output_dir, target_synsets, max_images_per_synset=1300, train_size=1000):
    os.makedirs(output_dir, exist_ok=True)
    
    # Create training and validation directories
    train_dir = os.path.join(output_dir, 'training')
    val_dir = os.path.join(output_dir, 'validation')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    
    start_time = time.time()
    total_extracted = 0
    
    for synset_id, animal_name in target_synsets.items():
        synset_tar_path = os.path.join(dataset_dir, f"{synset_id}.tar")
        
        # Create class output directories in both training and validation
        train_class_output_dir = os.path.join(train_dir, f"{synset_id}_{animal_name}")
        val_class_output_dir = os.path.join(val_dir, f"{synset_id}_{animal_name}")
        os.makedirs(train_class_output_dir, exist_ok=True)
        os.makedirs(val_class_output_dir, exist_ok=True)
        
        with tarfile.open(synset_tar_path, 'r') as tar:
            members = [m for m in tar.getmembers() if m.isfile()]
            members = members[:max_images_per_synset]
            
            for idx, member in enumerate(members):
                if idx < train_size:
                    # Copy to training directory
                    tar.extract(member, path=train_class_output_dir)
                else:
                    # Copy to validation directory
                    tar.extract(member, path=val_class_output_dir)
                
                total_extracted += 1
                
                if total_extracted >= len(target_synsets) * max_images_per_synset:
                    break
            
            if total_extracted >= len(target_synsets) * max_images_per_synset:
                break
    
    end_time = time.time()
    print(f"Extraction completed in {end_time - start_time:.2f} seconds")

# Define the dataset directory and the output directory
dataset_dir = '/home/ec2-user/SageMaker/imagenet_dataset'  
output_dir = '/home/ec2-user/SageMaker/imagenet_extracted'

# Extract images for target synsets, aiming for 1000 in training and 300 in validation per synset ID
extract_images_to_folders(dataset_dir, output_dir, target_species_synsets, max_images_per_synset=1300, train_size=1000)


2024-07-07 22:03:36.379601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 22:03:36.379874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 22:03:38.237233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-07 22:03:41.052812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-07 22:03:54.433476: W tensorflow/comp

Extraction completed in 55.10 seconds


In [ ]:
////////////////////////////////////////////////////////////////////////////////////////////////


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# Assuming img_height, img_width, and batch_size are defined
img_height = 224
img_width = 224
batch_size = 32

# Assuming class_names are defined
class_names = ['lion', 'tiger', 'zebra', 'elephant', 'hippo', 'meerkat', 
               'gorilla', 'king_penguin', 'cheetah', 'red_panda', 
               'black_bear', 'lemur', 'flamingo', 'chimpanzee', 
               'orangutan', 'jaguar', 'panda', 'armadillo', 'hyena', 'tree_frog']
num_classes = len(class_names)

# Check available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Define data directories
train_dir = '/home/ec2-user/SageMaker/imagenet_extracted/training'
val_dir = '/home/ec2-user/SageMaker/imagenet_extracted/validation'

# Load datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='int'
)

# Normalize datasets
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Load ResNet50 pretrained model without top layers
resnet50_model = tf.keras.applications.ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(img_height, img_width, 3)
)
resnet50_model.trainable = False

# Create the model
inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = resnet50_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(num_classes)(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1  # Set verbose to 1 for detailed progress bar
)


Found 20000 files belonging to 20 classes.
Found 6000 files belonging to 20 classes.
94765736/94765736 [==============================] - 2s 0us/step
Epoch 1/10


2024-07-07 22:10:23.375134: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27591300 exceeds 10% of free system memory.
2024-07-07 22:10:23.689285: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.
2024-07-07 22:10:23.711774: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 20313600 exceeds 10% of free system memory.
2024-07-07 22:10:23.735464: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-07-07 22:10:23.829363: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19267584 exceeds 10% of free system memory.


625/625 [==============================] - 3735s 6s/step - loss: 2.8792 - accuracy: 0.1144 - val_loss: 2.7652 - val_accuracy: 0.1790
Epoch 2/10
625/625 [==============================] - 3778s 6s/step - loss: 2.7352 - accuracy: 0.1767 - val_loss: 2.6951 - val_accuracy: 0.1857
Epoch 3/10
625/625 [==============================] - 3704s 6s/step - loss: 2.6583 - accuracy: 0.2074 - val_loss: 2.6443 - val_accuracy: 0.1927
Epoch 4/10
172/625 [=======>......................] - ETA: 33:08 - loss: 2.6031 - accuracy: 0.2269

In [ ]:
import matplotlib.pyplot as plt

# Assuming history contains the training history returned by model.fit()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
////////////////////////////////////////////////////////////////////////////////////////////////